In [1]:
import matplotlib
from matplotlib import font_manager, rcParams

rcParams["font.family"] = "NanumGothic"
rcParams["axes.unicode_minus"] = False 

In [2]:
from ultralytics import YOLO
from typing import Any
from pathlib import Path

# 데이터셋 YAML 경로
OUT_ROOT: Path = Path("yolo_transfer") # 필요시 경로 변경!!
day_yaml: Path = OUT_ROOT / "day_data" / "data.yaml"
night_yaml: Path = OUT_ROOT / "night_data" / "data.yaml"
badw_yaml: Path = OUT_ROOT / "bad_weather_data" / "data.yaml"

In [3]:
# 1단계: 낮/맑음
model_step1: YOLO = YOLO("yolo11m.pt")
results_step1: Any = model_step1.train(
    data=str(day_yaml),
    epochs=80,
    batch=6,                   
    imgsz=608,
    rect=True,
    optimizer="AdamW",
    lr0=1e-3,
    cos_lr=True,
    # close_mosaic=18,
    amp=True,
    workers=4,
    device="cuda",
    seed=0,
    freeze=10,
    patience=20,
    # 증강 관련
    # hsv_h=0.015,
    # hsv_s=0.5,
    # hsv_v=0.45,
    # degrees=1.0,
    # translate=0.06,
    # scale=0.25,
    # shear=0.03,
    # fliplr=0.2,
    # flipud=0.0,
    # mosaic=0.5,
    # mixup=0.05,
    # copy_paste=0.0,
    # perspective=0.0,
    # erasing=0.15,
    project="runs/day_stage",
    name="exp_day"
)

# 1단계 결과 경로
step1_last: str = "runs/day_stage/exp_day/weights/last.pt"
print(f"STEP1 last -> {step1_last}")

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.10.14 torch-2.3.1 CUDA:0 (Tesla V100-SXM2-32GB, 32494MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=yolo_transfer/day_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=608, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_day, nbs=64, nms=False, opse

In [ ]:
# 2단계: 야간 미세조정 (1단계 last 로드)
model_step2: YOLO = YOLO(step1_last)
results_step2: Any = model_step2.train(
    data=str(night_yaml),
    epochs=50,
    batch=8,
    imgsz=960,
    optimizer="AdamW",
    lr0=5e-5,
    cos_lr=True,
    close_mosaic=10,
    amp=True,
    workers=8,
    device="cuda",
    seed=0,
    freeze=10,
    patience=15,
    hsv_h=0.015,
    hsv_s=0.4,
    hsv_v=0.6,
    degrees=0.5,
    translate=0.03,
    scale=0.15,
    shear=0.0,
    fliplr=0.1,
    flipud=0.0,
    mosaic=0.2,
    mixup=0.0,
    copy_paste=0.0,
    erasing=0.1,
    project="runs/night_stage",
    name="exp_night"
)

# 2단계 결과 경로
step2_last: str = "runs/night_stage/exp_night/weights/last.pt"
print(f"STEP2 last -> {step2_last}")

In [1]:
# 3단계: 악천후 미세조정 (2단계 last 로드)
model_step3: YOLO = YOLO(step2_last)
results_step3: Any = model_step3.train(
    data=str(badw_yaml),
    epochs=40,
    batch=8,
    imgsz=960,
    optimizer="AdamW",
    lr0=3e-5,
    cosine_lr=True,
    close_mosaic=5,
    amp=True,
    workers=8,
    device="cuda",
    seed=0,
    freeze=10,
    patience=10,
    hsv_h=0.02,
    hsv_s=0.6,
    hsv_v=0.6,
    degrees=2.0,
    translate=0.1,
    scale=0.3,
    shear=0.05,
    fliplr=0.1,
    flipud=0.0,
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
    erasing=0.25,
    project="runs/bad_weather_stage",
    name="exp_bad_weather"
)

# 3단계 결과 경로
step3_last: str = "runs/bad_weather_stage/exp_bad_weather/weights/last.pt"
print(f"STEP3 last -> {step3_last}")

New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.10.14 torch-2.3.1 CUDA:0 (Tesla V100-SXM2-32GB, 32494MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_transfer/day_data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_day, nbs=64, nms=False, 

/opt/conda/lib/python3.10/site-packages/ultralytics/utils/plotting.py:628: UserWarning: Glyph 49849 (\N{HANGUL SYLLABLE SEUNG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/plotting.py:628: UserWarning: Glyph 50857 (\N{HANGUL SYLLABLE YONG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/plotting.py:628: UserWarning: Glyph 52264 (\N{HANGUL SYLLABLE CA}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/plotting.py:628: UserWarning: Glyph 49548 (\N{HANGUL SYLLABLE SO}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/opt/conda/lib/python3.10/site-packages/ultralytics/utils/plotting.py:628: UserWarning: Glyph 54805 (\N{HANGUL SYLLABLE HYEONG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/opt/conda/lib/python3.10/site-packages/ultral

optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /workspace/runs/day_stage/exp_day
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/80      25.7G     0.9507     0.7118     0.8972        185        640: 100% ━━━━━━━━━━━━ 4216/4216 4.1it/s 17:01<0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 295/295 5.5it/s 53.5s<0.2s
                   all       9425     125208       0.68      0.444      0.501      0.341

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/80      29.5G     0.9249     0.6439     0.8906        257        640: 50% ━━━━━━────── 2112/4216 8.3it/s 8:06<4:144


RuntimeError: !block->expandable_segment_ INTERNAL ASSERT FAILED at "/opt/conda/conda-bld/pytorch_1716905979055/work/c10/cuda/CUDACachingAllocator.cpp":2549, please report a bug to PyTorch. 